In [3]:
### 0. import
import psycopg2
import numpy  as np
import pandas as pd
from datetime import datetime
from selenium import webdriver
from bs4 import BeautifulSoup


### 1. 법정동명 가져오기 (법정동 3968개)
# AWS DW Connect
aws_conn = psycopg2.connect(
    host = 'redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
    port = 5439,
    dbname = 'dev',
    user = 'awsuser',
    password = 'cremaoAdmin1234qwer!!'
)
aws_cursor = aws_conn.cursor()

sql = "select distinct emd_nm, emd_cd from m1.emd"
aws_cursor.execute(sql)
aws_conn.commit()
rows = aws_cursor.fetchall()

df_emd = pd.DataFrame(rows)
df_emd.columns = ['emd_nm','emd_cd']
df_emd = df_emd.drop_duplicates('emd_nm')
df_emd['emd_cd'] = df_emd['emd_cd'] + '00'


### 2. 법정동별 게시물 수 수집 (1차 수집)
driver = webdriver.Chrome(executable_path='C:\oasis\git\pythonProject\업무 완료\driver\chromedriver.exe')
cnt_list = []
etc_emd_nm = [] # 추가작업 법정동

for dong in df_emd['emd_nm']:
    
    url = f'https://www.instagram.com/explore/tags/{dong}맛집/' # url로 태그 검색
    driver.get(url)
    time.sleep(3)
    
    try:
        text = driver.find_element_by_class_name('_ac2a').text # 전체 게시물 수 가져오기
        cnt_list.append(text)
        
    except:
        cnt = '0'
        cnt_list.append(cnt)
        etc_emd_nm.append(dong)
        time.sleep(3)
        
        pass

# 데이터프레임화
df_cnt_list = pd.DataFrame(cnt_list,columns=['cnt'])
df1 = pd.concat([df_emd,df_cnt_list],axis=1)


### 법정동별 게시물 수 수집 (2차 수집)
time.sleep(10)
cnt_list = [] # 초기화
dong_list = []
etc_dong_list = []

for dong in etc_emd_nm:
    
    url = f'https://www.instagram.com/explore/tags/{dong}맛집/' # url로 태그 검색
    driver.get(url)
    time.sleep(3)
    
    try:
        text = driver.find_element_by_class_name('_ac2a').text # 전체 게시물 수 가져오기
        cnt_list.append(text)
        dong_list.append(dong)
        
    except:
        cnt = '0'
        cnt_list.append(cnt)
        dong_list.append(dong)
        etc_dong_list.append(dong)
        
        time.sleep(3)
        pass

# 데이터프레임화
df2 = pd.DataFrame(data = zip(dong_list,cnt_list), columns = ['emd_nm','cnt'])
driver.close()

# 수집한 데이터 병합
df = pd.merge(df1,df2,on='emd_nm', how='left')
idx = df[df['cnt_y'].notnull()].index

for i in idx:
    df['cnt_x'][i] = df['cnt_y'][i]

# 숫자 , 제외 / base_ym 컬럼 추가
df = df.drop(columns=['cnt_y'])
df['cnt'] = df['cnt'].str.replace(',','')
df['base_ym'] = datetime.today().strftime("%Y%m")



### 3. DB INSERT
import datetime

table = 'm1.insta_emd_total_cnt'

# execute_mogrify
def execute_mogrify(conn, df, table, val):
    # Create a list of tuples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    cursor = conn.cursor()
    values = [cursor.mogrify(val, tup).decode('utf8') for tup in
              tuples]
    query = "INSERT INTO %s(%s) VALUES " % (table, cols) + ",".join(values)

    try:
        cursor.execute(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# values 설정
val = '(' + ('%s,' * len(df.columns))[:-1] + ')'

# insert
for j in range(0, len(df), 10000):

    # DB Connect
    conn = psycopg2.connect(
        host='redshift-cluster-1.ctvbwnnvbdkl.ap-northeast-2.redshift.amazonaws.com',
        port=5439,
        dbname='dev',
        user='awsuser',
        password='cremaoAdmin1234qwer!!'
    )

    # 10000개 단위로 나누기
    tmp = df[j:j + 10000]

    # insert
    execute_mogrify(conn, tmp, table, val)
    print(datetime.datetime.now(), ' : ', j)
    tmp = pd.DataFrame()

# row count
print(len(df))

NameError: name 'time' is not defined